<a href="https://colab.research.google.com/github/kabilan942/Legal/blob/main/Indian%20Legal%20Abs%20Ext/LDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import glob
from nltk import tokenize
import nltk
!pip install transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import torch.nn.functional as F
import torch
import json
import random
import os
import sys
sys.path.insert(0, '../')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 78.1 MB/s eta 0:00:00


## utilities.py

Defined Functions in utilities.py:

`get_root_path()`

`get_summary_data(dataset, train)`

`get_summary_data_rhet_train(dataset)
`
`get_req_len_dict(dataset, istrain)`

`split_to_sentences(para)`

`nest_sentencesV2(document,chunk_length)`

`nest_sentencesMV2(document_sents,chunk_length)`

`nest_sentencesV3(doc_sents,chunk_length, dict_sents_labels)`

`get_doc_sens_and_labels(doc)`

In [3]:
import glob
from nltk import tokenize
import nltk
!pip install transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch

def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    # path = "/home/pahelibhattacharya/Abhayv2/camera_ready/summarization/dataset"
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset"
    return path

def get_summary_data(dataset, train):
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''
    if dataset == "N2":
        path = get_root_path() + '/N2/Full-Text/India'
        all_files = glob.glob(path + "/*.txt")

        data_source = []
        names = []
        for filename in all_files:
            with open(filename, 'r') as f: 
                p = filename.rfind("/")
#                 print(filename[p+1:])
                names.append(filename[p+1:])
                a = f.read()
                data_source.append(a)
        return names, data_source, []
    
    path = get_root_path() + '/Summary-Data-' + dataset + '/' + train + '-data/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/Summary-Data-' + dataset + '/' + train + '-data/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f: 
            a = f.read()
            l = len(a)
            data_summary.append(a)
            
    return names, data_source, data_summary

def get_summary_data_rhet_train(dataset):
    '''
    function to get names, documents, and summaries for Rhetorical labeled documents - train
    change the path variable to the path of the Rhetorical labeled dataset
    '''
    path = get_root_path() + '/rhet/' + dataset.lower() + '_ft_rhet' # use your path
    all_files = glob.glob(path + "/*.txt")

    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f: 
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)

    path = get_root_path() + '/rhet/RhetSumm_Dataset/raw_files/'+ dataset +'/summary' # use your path
    all_files = glob.glob(path + "/*.txt")

    data_summary = {}
    for filename in all_files:
        with open(filename, 'r') as f: 
            p = filename.rfind("/")
            a = f.read()
            l = len(a)
            data_summary[filename[p+1:]] = (a)
    return names, data_source, data_summary

def get_summary_data_rhet_test(dataset):
    '''
    function to get names, documents, and summaries for Rhetorical labeled documents - test
    change the path variable to the path of the Rhetorical labeled dataset
    '''
    path = get_root_path() + '/rhet/RhetSumm_Dataset/rhet/' + dataset + "/" # use your path
    all_files = glob.glob(path + "/*.txt")

    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f: 
            p = filename.rfind("/")
#             print(filename[p+1:])
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)

    return names, data_source


def get_req_len_dict(dataset, istrain):
    '''
    function to required length data for each summary
    change the path variable to the path to Summary_Length_India.txt/ Summary_Length_Uk.txt file
    '''
    
    if dataset == "N2":
        f = open(get_root_path() + "/N2/Summary_Length_India.txt", "r")
        a = (f.read())
        a = a.split("\n")
        dict_names = {}
        for i in a:
            b = i.split("	")
            dict_names[b[0] + ".txt"] = int(b[1])
        return dict_names 
    
    f = open(get_root_path() + "/Summary-Data-"+ dataset +"/length-file-" + istrain +".txt", "r")
    a = (f.read())
    a = a.split("\n")
    dict_names = {}
    for i in a:
        b = i.split("	")
        try:
            tp = int(b[2])
            dict_names[b[0]] = tp
        except:
            print(b)
    return dict_names  

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)
    return sents

def nest_sentencesV2(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a list of sentences.
    '''
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(sent)
    return nested

def nest_sentencesMV2(document_sents,chunk_length):
    '''
    function to chunk a document
    input:  doc_sents           - Input document sentences
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a list of sentences.
    '''
    nested = []
    sent = []
    length = 0
    #modeified v2
    
    for sentence in document_sents:
        length += len((sentence.split(" ")))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(sent)
    return nested

def nest_sentences(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a string.
    '''
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(" ".join(sent))
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(" ".join(sent))
    return nested
  

def nest_sentencesV3(doc_sents,chunk_length, dict_sents_labels):
    '''
    function to first segment the document using rhetorical roles and then chunk if required
    input:  doc_sents           - Input document sentences
            chunk_length        - chunk length
            dict_sents_labels   - dictionary for every sentence and label
    output: list of chunks
    '''
    s = list(set(dict_sents_labels.values()))
#     print(s)
    all_chunks = []
    
    for label in s:
        doc_sents_withlabels = []
        for sent in doc_sents:
            if sent == '':continue
            if dict_sents_labels[sent] == label:
                doc_sents_withlabels.append(sent)
        chunks = nest_sentencesMV2(doc_sents_withlabels, chunk_length)
        
        edited_chunks = []
        for chunk in chunks:
            edited_chunks.append(["<" + label + ">"] + chunk)
        
        all_chunks = all_chunks + edited_chunks

    return all_chunks   

def get_doc_sens_and_labels(doc):
    '''
    Function to read a Rhetorically labeled document.
    returns a list of sentences, the label of each sentence, dictionary: keys-sentences, values-labels
    '''
    sents = []
    labels = []
    dict_sents_labels = {}
    ss = doc.split("\n")
    for i in ss:
        try:
            spt = i.split("\t")
            sents.append(spt[0])
            labels.append(spt[1])
            dict_sents_labels[spt[0]] = spt[1] 
        except:
            pass
    return sents, labels, dict_sents_labels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Chunker.ipynb
Script to chunk and save every file. Every line in the output file is a chunk.


In [11]:
def get_root_path():
    # path = "/home/pahelibhattacharya/Abhayv2/camera_ready/summarization/dataset"
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs"
    return path

def get_summary_data(dataset, train):  
    path = get_root_path() + '/' + train + '-data/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/' + train + '-data/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f: 
            a = f.read()
            l = len(a)
            data_summary.append(a)
            
    return names, data_source, data_summary

import nltk
nltk.download('punkt')

def nest_sentences(document,chunk_length):
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(" ".join(sent))
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(" ".join(sent))
    return nested

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
dataset = "IN" # Options: IN - IN-Abs, UK-UK-Abs, N2-IN-Ext 
# output_path = "./output/"
output_path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data"

names, data_source, data_summary = get_summary_data(dataset, "test")    # names, documents and summaries

print(len(names))
print(len(data_source))
print(len(data_summary))


100
100
100


In [13]:
names[0]

'3210.txt'

In [14]:
data_source[0]

'Appeal No. 1604 of 1967.\nFrom the judgment and order dated the 8th February, 1967 of Le Delhi High Court in Civil Writ No. 531 D of 1964.\nP.P. Rao and R. N. Sachthey, for the appellants.\nG.P. Pai, P. C. Bhartari and O. C. Mathur, for respondents nos.\n1 & 2.\nThe Judgment of the Court was delivered by SARKARIA J.\nWhether the Federation of Indian Chambers of Commerce and Industry, New Delhi (Respondent 1) is a "commercial establishment" within the meaning of s.2(5) of the Delhi Shops ,and Establishments Act, 1954 (for short the Act) is the sale question that falls for determination in this appeal by certificate directed against the judgment dated February 8, 1967 of the High Court of Delhi.\nThe facts bearing on this question may now be stated: Respondent 1 (hereinafter referred to as the Federation) is a Company registered under section 26 of the Indian Companies Act, 1913.\nThe primary objects of the Federation as given in the Memorandum of Association are "(a) To promote Indian 

In [15]:
len(data_source[0])

22885

In [16]:
data_summary[0]

"The respondent was required by the Chief Inspector of Shops and Establishments to register its establishment ' under the Delhi Shops and Establishments Act, 1954.\nThe respondent contended that it was not an 'establishment ' which is defined in section 2(9) as meaning 'a shop or a commercial estab lishment ' and did not comply with the direction.\nProceedings for prosecution of its Secretary were instituted, where upon the respondent filed a petition in the High Court for quashing the order of the appellant and for directing the Magistrate not to proceed with the complaint.\nBefore the High Court the appellant contended that the respondent was a 'commercial establishment ' because, the activity of the respondent amounted to a 'Profession ', and alternatively, that the case fell within the latter part of the definition of 'commercial establishment ' in section 2(5), inasmuch as its activities were connected with trade and business generally, but, it was not urged that the activity of t

In [17]:
len(data_summary[0])

3560

In [ ]:
import os
if not os.path.exists(output_path + '/'+ dataset +'_chunked/'):
    os.makedirs(output_path + '/'+ dataset +'_chunked/')

for i in range(len(data_source)):
    doc = data_source[i].replace("\n", " ")
    chunks = nest_sentences(doc, 512)
    save = "\n".join(chunks)
    path = output_path +  "/"+ dataset +"_chunked/" + names[i]
    file = open(path,'w')
    file.write(save)
    file.close()

## Pointer Generator

### Chunker

Each chunk would have its own text file

In [ ]:
# output_path = "./chunked/"
output_path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
def get_root_path():
    # path = "/home/pahelibhattacharya/Abhayv2/camera_ready/summarization/dataset"
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs"
    return path

def get_summary_data(dataset, train):  
    path = get_root_path() + '/' + train + '-data/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/' + train + '-data/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f: 
            a = f.read()
            l = len(a)
            data_summary.append(a)
            
    return names, data_source, data_summary

import nltk
nltk.download('punkt')

def nest_sentences(document,chunk_length):
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(" ".join(sent))
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(" ".join(sent))
    return nested

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
names, data_source, data_summary = get_summary_data("IN", "test")
print(len(names))
print(len(data_source))
print(len(data_summary))

100
100
100


In [ ]:
names[0]

'3210.txt'

In [ ]:
data_source[0]

'Appeal No. 1604 of 1967.\nFrom the judgment and order dated the 8th February, 1967 of Le Delhi High Court in Civil Writ No. 531 D of 1964.\nP.P. Rao and R. N. Sachthey, for the appellants.\nG.P. Pai, P. C. Bhartari and O. C. Mathur, for respondents nos.\n1 & 2.\nThe Judgment of the Court was delivered by SARKARIA J.\nWhether the Federation of Indian Chambers of Commerce and Industry, New Delhi (Respondent 1) is a "commercial establishment" within the meaning of s.2(5) of the Delhi Shops ,and Establishments Act, 1954 (for short the Act) is the sale question that falls for determination in this appeal by certificate directed against the judgment dated February 8, 1967 of the High Court of Delhi.\nThe facts bearing on this question may now be stated: Respondent 1 (hereinafter referred to as the Federation) is a Company registered under section 26 of the Indian Companies Act, 1913.\nThe primary objects of the Federation as given in the Memorandum of Association are "(a) To promote Indian 

In [ ]:
data_summary[0]

'Appeal No. 1604 of 1967.\nFrom the judgment and order dated the 8th February, 1967 of Le Delhi High Court in Civil Writ No. 531 D of 1964.\nP.P. Rao and R. N. Sachthey, for the appellants.\nG.P. Pai, P. C. Bhartari and O. C. Mathur, for respondents nos.\n1 & 2.\nThe Judgment of the Court was delivered by SARKARIA J.\nWhether the Federation of Indian Chambers of Commerce and Industry, New Delhi (Respondent 1) is a "commercial establishment" within the meaning of s.2(5) of the Delhi Shops ,and Establishments Act, 1954 (for short the Act) is the sale question that falls for determination in this appeal by certificate directed against the judgment dated February 8, 1967 of the High Court of Delhi.\nThe facts bearing on this question may now be stated: Respondent 1 (hereinafter referred to as the Federation) is a Company registered under section 26 of the Indian Companies Act, 1913.\nThe primary objects of the Federation as given in the Memorandum of Association are "(a) To promote Indian 

In [ ]:
output = []
for i in range(len(data_source)):
    name = names[i]
    doc = data_source[i]
    wc = doc.split(" ")
    print(str(i) + ": " + name)
    nested = nest_sentences(doc,395)
    
    for i in range(len(nested)):
        path = output_path + name[:-4] + "_" + str(i) + ".txt"
        print(path)
        file = open(path,'w')
        file.write(nested[i])
        file.close()
    
print(output)

0: 3210.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_0.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_1.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_2.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_3.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_4.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_5.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_6.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_7.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_8.txt
/content/drive/MyDrive/Le

The 100 documents in test data of IN-Abs has been chunked into several docs with max length of each doc being 395 words.

## Legal LED

### Fine-Tuning 

In [7]:
from IPython.display import display, HTML
import torch
!pip install datasets
from datasets import load_dataset, load_metric, Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import LEDTokenizer, LEDForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [6]:
val_files = [] # Add the validation files to be used

In [8]:
model_name = "nsi319/legal-led-base-16384"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [9]:
def getData(tokenizer, dataPath, MAX_DOC_LEN, val = 0):
	documentPath = f'{dataPath}/judgement'     # path for judgement of train data in IN-Abs
	summaryPath = f'{dataPath}/summary'        # path for summary of train data in IN-Abs
	dataset = {'document':[], 'summary':[]}    # dictionary with document and summary for creating dataframe
	count = 0
	for file in os.listdir(documentPath):      # used to get the list of all files and directories in the specified directory - here judgement in train data
		count += 1
		if os.stat(f'{documentPath}/{file}').st_size == 0 or os.stat(f'{summaryPath}/{file}').st_size == 0:
			continue			
		doc_in = open(f'{documentPath}/{file}', 'r', encoding='utf8')
		doc_lines = [line.strip() for line in doc_in.readlines()]
		summ_in = open(f'{summaryPath}/{file}', 'r', encoding='utf8')
		summ_lines = [line.strip() for line in summ_in.readlines()]
		if len(doc_lines) == 0 or len(summ_lines) == 0:
			continue

		# print(file, train_files[0], type(file))
		if val == 0 and file not in val_files:     # for train_df
			dataset['document'].append(' '.join(doc_lines))
			dataset['summary'].append(' '.join(summ_lines))
		if val == 1 and file in val_files:         # for val_df
			dataset['document'].append(' '.join(doc_lines))
			dataset['summary'].append(' '.join(summ_lines))
	
	df = pd.DataFrame(dataset)
	return df

In [10]:
exp = 'exp1'
encoder_max_length = 1024*16
decoder_max_length = 1024
batch_size = 1
n_epochs = 3

In [11]:
dataPath = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs"
train_df = getData(tokenizer, f'{dataPath}/train-data', encoder_max_length-2)  # tokenizer = AutoTokenizer.from_pretrained(model_name)
train_dataset = Dataset.from_pandas(train_df)                                       # model_name = "nsi319/legal-led-base-16384"
val_df = getData(tokenizer, f'{dataPath}/train-data', encoder_max_length-2,1)   # 4th argument -> value = 1
val_dataset = Dataset.from_pandas(val_df)

In [13]:
train_df.head()

,document,summary
0,ISDICTION: Civil Appeal No. 855 (n) of 1979. F...,In Civil Appeal No. 855(N) of 1979 and Civil A...
1,ivil Appeal No. 1369 of 1990. From the Judgmen...,"The question for determination is, can a licen..."
2,vil Appeal No. 277 1 of 1981. From the Judgmen...,The Respondent landlord filed a suit under the...
3,iminal Appeals Nos. 102 and 103 of 1958. Appea...,The two appellants and one other person were a...
4,vil Appeals Nos. 596 597 (NL) of 1986. From th...,"Under the , the State Government referred to t..."


In [14]:
val_df.head()

,document,summary


Preparing and loading the final datasets

In [12]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs (documents)
    inputs = tokenizer(
        batch["document"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,  # = 16384
    )
    # tokenize th labels (summary)
    outputs = tokenizer(
        batch["summary"],
        padding="max_length",
        truncation=True,
        max_length=decoder_max_length,  # = 1024
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [15]:
# map train data
train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,   # batch_size = 1
    remove_columns=["document", "summary"],
)


  0%|          | 0/7028 [00:00<?, ?ba/s]

In [16]:
# set Python list to PyTorch tensor
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

No elements in validation dataset?

In [17]:
# map val data
val_dataset = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["document", "summary"],
)


# set Python list to PyTorch tensor
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

ValueError: ignored

**Training**

In [1]:
!pip install nltk
import nltk
nltk.download('punkt')
!pip install rouge_score
!pip install datasets evaluate transformers[sentencepiece]
import evaluate

# load rouge
rouge = evaluate.load("rouge")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=12e502669595f386d83887ecffa7a007ed36adccd1f88cd433d232cf5e7fd2d9
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 2

In [2]:
def postprocess_text(preds, labels):
	preds = [pred.strip() for pred in preds]
	labels = [label.strip() for label in labels]

	# rougeLSum expects newline after each sentence
	preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
	labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

	return preds, labels


def compute_metrics(pred):
	labels_ids = pred.label_ids
	pred_ids = pred.predictions

	pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
	labels_ids[labels_ids == -100] = tokenizer.pad_token_id
	label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

	# Some simple post-processing
	pred_str, label_str = postprocess_text(pred_str, label_str)
	
	result = rouge.compute(
		predictions=pred_str, references=label_str, use_stemmer=True
	)

	# Extract a few results from ROUGE
	result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

	prediction_lens = [
		np.count_nonzero(pred != tokenizer.pad_token_id) for pred in pred_ids
	]
	result["gen_len"] = np.mean(prediction_lens)
	result = {k: round(v, 4) for k, v in result.items()}
	
	return result

Training Arguments

In [ ]:
training_args = Seq2SeqTrainingArguments(
	output_dir=f"/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/Abs Methods/Legal-LED/final/{exp}",
	num_train_epochs=n_epochs,
	per_device_train_batch_size=batch_size,
	per_device_eval_batch_size=batch_size,
	fp16=True,
	evaluation_strategy="epoch",
	save_strategy="epoch",
	load_best_model_at_end=True,
	metric_for_best_model="eval_rouge2",
	greater_is_better=True,
	warmup_steps=200,
	predict_with_generate=True,
	logging_dir=f"led_logs/final/{exp}",
	logging_steps=50,
    gradient_accumulation_steps=4,
	save_total_limit=1 #save only the best model
)

Define Model

In [ ]:
# load model + enable gradient checkpointing & disable cache for checkpointing
led = AutoModelForSeq2SeqLM.from_pretrained(model_name, gradient_checkpointing=True, use_cache=False)
# led.resize_token_embeddings(len(tokenizer))

# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = decoder_max_length
led.config.min_length = 256
# led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 4

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()

#Save the finetuned model
# model_checkpoint_dir = f"results/led/{exp}/best_model"
# trainer.save_model(model_checkpoint_dir)

trainer.save_model("/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/Abs Methods/Legal-LED/final_model/IN_model")

### Generating Summaries

In [ ]:
dataset = "IN" # Options: IN - IN-Abs, UK-UK-Abs, N2-IN-Ext
output_path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/Abs Methods/Legal-LED"  # output path to save the model in drive

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
def get_root_path():
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset"
    return path


def get_req_len_dict(dataset, istrain):
    '''
    function to get required length data for each summary
    change the path variable to the path to Summary_Length_India.txt/ Summary_Length_Uk.txt file
    '''
    
    if dataset == "N2":
        f = open(get_root_path() + "/N2/Summary_Length_India.txt", "r")
        a = (f.read())
        a = a.split("\n")
        dict_names = {}
        for i in a:
            b = i.split("	")
            dict_names[b[0] + ".txt"] = int(b[1])
        return dict_names 
    
    f = open(get_root_path() + "/IN-Abs/" + istrain + "/-data/summary" +".txt", "r")
    a = (f.read())
    a = a.split("\n")
    dict_names = {}
    for i in a:
        b = i.split("	")
        try:
            tp = int(b[2])
            dict_names[b[0]] = tp
        except:
            print(b)
    return dict_names

In [ ]:
#Reading the test documents
names, data_source, data_summary = get_summary_data(dataset, "test")
print(len(names))
print(len(data_source))
print(len(data_summary))
len_dic = dict_names = get_req_len_dict(dataset, "test")

In [ ]:
model_name = "nsi319/legal-led-base-16384"
tokenizer = AutoTokenizer.from_pretrained(model_name)

encoder_max_length = 1024*16      # = 16384 (input sequence length) 
decoder_max_length = 1024

In [ ]:
led = AutoModelForSeq2SeqLM.from_pretrained(model_name, gradient_checkpointing=True, use_cache=False)
# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = decoder_max_length
led.config.min_length = 256
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 4


In [ ]:
led = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/Abs Methods/Legal-LED/final_model/IN_model", 
                                            use_cache=False)
led = led.to("cuda")

In [ ]:
import torch
led.eval()

def generate_summary_gpu(input_text, l):
	inputs = tokenizer(
		input_text,
		padding="max_length",  # = decoder_max_length = 1024
		truncation=True,
		max_length=encoder_max_length,  # = 16384
		return_tensors="pt",
	)
	input_ids = inputs.input_ids.to(led.device)
	attention_mask = inputs.attention_mask.to(led.device)
	global_attention_mask = torch.zeros_like(attention_mask)
	# put global attention on <s> token
	global_attention_mask[:, 0] = 1
	l = max(l,256)
	l = min(l,1024)
	outputs = led.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask, max_length = l, num_beams = 2,repetition_penalty = 2.5,early_stopping = True)	
	preds = [tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_space=True) for gen_id in outputs]
	return "".join(preds)


In [ ]:
for i in range(len(data_source)):
    name = names[i]
    doc = data_source[i]
    wc = doc.split(" ")
    input_len = len(wc)
    req_len = dict_names[name]
    print(str(i) + ": " + name +  " - " + str(input_len) + " : " + str(req_len))
    
    abs_summ = generate_summary_gpu(doc,req_len)
    if len(abs_summ.split(" ")) > req_len:
        abs_summ = abs_summ.split(" ")
        abs_summ = abs_summ[:req_len]
        abs_summ = " ".join(abs_summ)
    print(abs_summ)
    print(len((abs_summ.split(" "))))
    path = output_path + name
    file = open(path,'w')
    file.write(abs_summ)
    file.close()